# 本金平均攤還試算

## 前言

貸款的攤還方式主要有兩種：
- 本息平均攤還：每月繳款金額（本金 + 利息）相同
- 本金平均攤還：每月繳款本金相同，利息隨剩餘貸款金額改變

## 實作

本次作業實作**本金平均攤試算**，目標做出與[台南第三信用合作社](https://ttc.scu.org.tw/memdca1.htm)的試算結果相同的程式，以下將分為四個步驟處理：
- Step 1: 資料輸入
- Step 2: 本金計算
- Step 3: 利息計算
- Step 4: 資料輸出


### Step 1: 資料輸入

提供給使用者輸入的資料有三項：
1. 本金：以萬元為輸入單位

In [ ]:
principal = float(input("本金(萬元): ")) * 10000

2. 期數：以年為輸入單位，因此需要換算成月份數量

In [ ]:
period = int(input("期數(年): ")) * 12

3. 年利率：以百分比為輸入單位

In [ ]:
r = float(input("年利率(%): "))

### Step 2: 本金計算

本金的計算方式為：<br>
**每月攤還本金(元) = 本金(元) / 期數(月)**<br>
若計算出來的結果包含小數，則無條件進位到整數

In [ ]:
avg_payment = math.ceil(principal / period)

也因為每月攤還本金的計算方式採無條件進位，因此最後一期的攤還本金只需要繳納剩餘的本金即可

In [ ]:
payment = avg_payment if remaining_principal > avg_payment else int(remaining_principal)

### Step 3: 利息計算

利息的計算方式為：<br>
**每月攤還利息(元) = 剩餘本金(元) * 月利率(％)**<br>
若計算出來的結果包含小數，則四捨五入到整數
(注意：python3 的 `round()` 並非四捨五入，詳情請參考[官方文件](https://docs.python.org/3/library/functions.html#round))

In [ ]:
decimal, integer = math.modf(remaining_principal * r / 1200)
interest = int(integer + 1) if decimal >= 0.5 else int(integer)

### Step 4: 資料輸出

為了方便及美觀，我使用 `pandas` 及 `tabulate` 兩個套件進行資料的儲存與輸出

In [ ]:
table = pd.DataFrame({"本金(元)": payments, "利息(元)": interests, "本金利息累計(元)": accumulated}, index=indices)
summary = pd.DataFrame(data={"": [int(principal), int(period/12), r, avg_payment, table["利息(元)"].sum()]}, index=["本金(元)", "期數(年)", "年利率(%)", "平均每月攤還本金(元)", "全部利息(元)"], dtype='object')

print("\n")
print("基本資訊:")
print(tabulate(summary, tablefmt="grid"))
print("\n")
print(f"每月攤還本金、利息一覽表(共 {int(period)} 期):")
print(tabulate(table, headers='keys', tablefmt="grid"))

## 成果

完整程式碼：

In [5]:
import math
import pandas as pd
from tabulate import tabulate


principal = float(input("本金(萬元): ")) * 10000
period = int(input("期數(年): ")) * 12
r = float(input("年利率(%): "))

avg_payment = math.ceil(principal / period)
remaining_principal = principal
payments, interests, accumulated, indices  = [list() for i in range(4)]

total = 0
for i in range(period):
    payment = avg_payment if remaining_principal > avg_payment else int(remaining_principal)
    decimal, integer = math.modf(remaining_principal * r / 1200)
    interest = int(integer + 1) if decimal >= 0.5 else int(integer)
    total += payment + interest
    
    payments.append(payment)
    interests.append(interest)
    accumulated.append(total)
    indices.append(f"第 {i+1} 期")
    
    remaining_principal -= payment
    
table = pd.DataFrame({"本金(元)": payments, "利息(元)": interests, "本金利息累計(元)": accumulated}, index=indices)
summary = pd.DataFrame(data={"": [int(principal), int(period/12), r, avg_payment, table["利息(元)"].sum()]}, index=["本金(元)", "期數(年)", "年利率(%)", "平均每月攤還本金(元)", "全部利息(元)"], dtype='object')

print("\n")
print("基本資訊:")
print(tabulate(summary, tablefmt="grid"))
print("\n")
print(f"每月攤還本金、利息一覽表(共 {int(period)} 期):")
print(tabulate(table, headers='keys', tablefmt="grid"))

本金(萬元):  15
期數(年):  1
年利率(%):  6




基本資訊:
+----------------------+--------+
| 本金(元)             | 150000 |
+----------------------+--------+
| 期數(年)             |      1 |
+----------------------+--------+
| 年利率(%)            |      6 |
+----------------------+--------+
| 平均每月攤還本金(元) |  12500 |
+----------------------+--------+
| 全部利息(元)         |   4878 |
+----------------------+--------+


每月攤還本金、利息一覽表(共 12 期):
+----------+------------+------------+--------------------+
|          |   本金(元) |   利息(元) |   本金利息累計(元) |
+==========+============+============+====================+
| 第 1 期  |      12500 |        750 |              13250 |
+----------+------------+------------+--------------------+
| 第 2 期  |      12500 |        688 |              26438 |
+----------+------------+------------+--------------------+
| 第 3 期  |      12500 |        625 |              39563 |
+----------+------------+------------+--------------------+
| 第 4 期  |      12500 |        563 |              52626 |
+----------+------------+------------+

## TODO
- [ ] 實作本息平均攤還，提供使用者另一個選項
- [ ] 確保輸入資料格式、型態正確，並提供防呆提醒